In [6]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-123220")
exp = Experiment(workspace=ws, name="hdrive")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-123220
Azure region: southcentralus
Subscription id: 30d182b7-c8c4-421c-8fa0-d3037ecfe6d2
Resource group: aml-quickstarts-123220


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Choose a name for your CPU cluster
# Use vm_size = \"Standard_D2_V2\" in your provisioning configuration.
cpu_cluster_name = "hyper-drive"

# max_nodes should be no greater than 4.

# Reference - https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                               max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output= True)
print("Azure Machine Learning Compute attached")

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Azure Machine Learning Compute attached


In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler

ps = RandomParameterSampling( {

        "--C": choice(100, 10, 1.0, 0.1, 0.01),
        "--max_iter": choice(25, 50, 100,125)
    })

# Specify a Policy
policy = BanditPolicy(evaluation_interval=5, slack_factor=0.20)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py

est = SKLearn(source_directory=".", 
              compute_target=compute_target, 
              entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.

hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name="Accuracy", 
                                     estimator=est, policy=policy,
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs=50, 
                                     max_concurrent_runs=5)

In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

#show run details with widget
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [12]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()

best_model= best_run.register_model(model_name='hyperdrive_model', model_path='./')

### YOUR CODE HERE ###



In [13]:
print(best_model)

Model(workspace=Workspace.create(name='quick-starts-ws-123220', subscription_id='30d182b7-c8c4-421c-8fa0-d3037ecfe6d2', resource_group='aml-quickstarts-123220'), name=hyperdrive_model, id=hyperdrive_model:1, version=1, tags={}, properties={})


In [14]:
#print best parameters and accuracy

print("Best Run Id: ", best_run.id)
print('\n Accuracy:', best_run.get_metrics()['Accuracy'])

print("\n Parameters: ", best_run.get_details()['runDefinition']['arguments'])

Best Run Id:  HD_33f7854c-ceda-4594-8e97-99a980d7dfb9_1

 Accuracy: 0.9092330329225676

 Parameters:  ['--C', '100', '--max_iter', '50']


In [ ]:
compute_target.delete()